<h1><center>Assignment 1<center><h1>


<h2><center>Kevin Thomas<center><h2>

<h3>Importing Libraries, Gathering Data and Preprocessing data<h3>

Importing nltk

In [1]:
import nltk
from nltk.corpus import udhr
from nltk import ngrams, FreqDist, ConditionalFreqDist
from nltk.tokenize import RegexpTokenizer
import math
from collections import defaultdict, Counter

Creating datasets for English, French, Italian and Spanish

In [2]:
eng = udhr.raw('English-Latin1')
fre = udhr.raw('French_Francais-Latin1')
ita = udhr.raw('Italian_Italiano-Latin1')
spa = udhr.raw('Spanish_Espanol-Latin1')

Splitting datasets into training and development sections

In [3]:
eng_train, eng_dev = eng[0:1000], eng[1000:1100]
fre_train, fre_dev = fre[0:1000], fre[1000:1100]
ita_train, ita_dev = ita[0:1000], ita[1000:1100]
spa_train, spa_dev = spa[0:1000], spa[1000:1100]

Creating test sets from the data

In [4]:
eng_test = udhr.words('English-Latin1')[0:1000]
fre_test = udhr.words('French_Francais-Latin1')[0:1000]
ita_test = udhr.words('Italian_Italiano-Latin1')[0:1000]
spa_test = udhr.words('Spanish_Espanol-Latin1')[0:1000]

In [5]:
eng_dev = eng_dev.split()
fre_dev = fre_dev.split()
ita_dev = ita_dev.split()
spa_dev = spa_dev.split()

In [6]:
tokenizer = RegexpTokenizer("[a-zA-Z'`éèî]+")

<h2>Problem 1: Language Model Creation<h2>

<h4>Creating an English vs. French language models<h4>

<h3>Unigram Classification<h3>

Creating unigram models based on English and French training data and comparing their accuracy when tested on Test data taken from the English text data. This is done by using the ngrams function from NLTK

In [7]:
#Function to create unigrams from data
def make_unigrams(text):
    text = text.lower()
    text = tokenizer.tokenize(text)
    text_unigrams = []
    for word in text:
        text_unigrams.append(list(ngrams(word, 1)))
    text_unigrams = [word for sublist in text_unigrams for word in sublist]
    text_final = text_unigrams
    for idx, val in enumerate(text_unigrams):
        text_final[idx] = ''.join(val)
    return text_final

In [8]:
#Creating unigram model for English and French training data
eng_uni = make_unigrams(eng_train)
fre_uni = make_unigrams(fre_train)

In [9]:
#Finding the frequency distribution of the created unigrams
eng_uni_freq = FreqDist(eng_uni)
fre_uni_freq = FreqDist(fre_uni)

The unigram probability of a word is calculated by taking the product of all the characters in the word. The probabilites of the characters can be found by dividing the frequency of that character in the training data by the total number of characters in the training data

In [10]:
#Function to estimate the probability of words in the test data based on the unigram models created on training data
def find_uni_prob(freq, test):
    total = 0
    for word in freq:
        total += freq[word]
    probs = []
    total+=1
    for word in test:
        word = word.lower()
        word_prob = 1
        for c in word:
            #if c in freq:
            word_prob *= (freq[c]/total)
        probs.append(word_prob)
    return probs

In [11]:
#Finding the probabilities of words in the english test dataset
eng_uni_prob = find_uni_prob(eng_uni_freq, eng_test)
fre_uni_prob = find_uni_prob(fre_uni_freq, eng_test)

Accuracy is calculated by scoring the number of words for which a language model has given a higher probability

In [12]:
#Comparing the probabilities of each word in the development set, creating scores for the models and calculating accuracies
eng_uni_score = fre_uni_score = 0
for i in range(len(eng_uni_prob)):
    if eng_uni_prob[i]>fre_uni_prob[i]:
        eng_uni_score+=1
    else:
        fre_uni_score+=1

eng_uni_acc = (eng_uni_score/len(eng_uni_prob))*100
fre_uni_acc = (fre_uni_score/len(eng_uni_prob))*100

<h3>Bigram Classification<h3>

<p>Bigrams consist of two character components found in the training data. They are created in a similar manner as unigrams, i.e. by using the ngrams method of NLTK. Additionally, the bigrams are padded with the "_" symbol to account for word beginning and endings</p>

In [13]:
#Function to create bigrams
def make_bigrams(text):
    text = text.lower()
    text = tokenizer.tokenize(text)
    text_bigrams = []
    for word in text:
        text_bigrams.append(list(ngrams(word, 2, pad_left=True, pad_right=True, left_pad_symbol="_", right_pad_symbol="_")))
    text_bigrams = [word for sublist in text_bigrams for word in sublist]
    text_final = text_bigrams
    for idx, val in enumerate(text_bigrams):
        text_final[idx] = ''.join(val)
    return text_final

In [14]:
#Creating Bigrams for English and French training data
eng_bi = make_bigrams(eng_train)
fre_bi = make_bigrams(fre_train)

In [15]:
#Creating freqency distributions of English and French
eng_bi_freq = FreqDist(eng_bi)
fre_bi_freq = FreqDist(fre_bi)

To find bigram probabilities, we find the product of the individual bigram probabilities in the word. The individual bigram probabilities can be calculated by taking the the frequency of the bigram in the testing data and dividing it by the unigram frequency of the first character.<br>
Additionally, to account for unknown bigrams, Laplace or Add-One smoothing is performed. To do this, the numerator is increased by one, and the size of the vocabulary (number of unique characters in the training data) is added to the denominator<br>

In [16]:
#Function to find the bigram probability of the test data
def find_bi_prob(uni, freq_uni, freq_bi, test):
    v = len(set(uni))
    probs = []
    for word in test:
        word = word.lower()
        #word = word
        word_prob = 1
        for c in range(len(word)-1):
            word_prob *= ((freq_bi[word[c:c+2]]+1)/(freq_uni[word[c]]+v))
        probs.append(word_prob)
    return probs

In [17]:
#Finding the bigram probability of test data
eng_bi_prob = find_bi_prob(eng_uni, eng_uni_freq, eng_bi_freq, eng_test)
fre_bi_prob = find_bi_prob(fre_uni, fre_uni_freq, fre_bi_freq, eng_test)

In [18]:
#Comparing the probabilities of each word in the test set, creating scores for the models and calculating accuracies
eng_bi_score = fre_bi_score = 0
for i in range(len(eng_bi_prob)):
    if eng_bi_prob[i]>fre_bi_prob[i]:
        eng_bi_score+=1
    else:
        fre_bi_score+=1

eng_bi_acc = (eng_bi_score/len(eng_bi_prob))*100
fre_bi_acc = (fre_bi_score/len(eng_bi_prob))*100

<h3>Trigram Classification<h3>

<p>Bigrams consist of tree character components found in the training data. They are created in a similar manner as unigrams, i.e. by using the ngrams method of NLTK. Additionally, the bigrams are padded with the "_" symbol to account for word beginning and endings</p>

In [19]:
#Function to create trigrams from training data
def make_trigrams(text):
    text = text.lower()
    text = tokenizer.tokenize(text)
    text_trigrams = []
    for word in text:
        text_trigrams.append(list(ngrams(word, 3, pad_left=True, pad_right=True, left_pad_symbol="_", right_pad_symbol="_")))
    text_trigrams = [word for sublist in text_trigrams for word in sublist]
    text_final = text_trigrams
    for idx, val in enumerate(text_trigrams):
        text_final[idx] = ''.join(val)
    return text_final

In [20]:
#Creating bgrams from training data
eng_tri = make_trigrams(eng_train)
fre_tri = make_trigrams(fre_train)

In [21]:
#Creating frequency distributions from the trigrams
eng_tri_freq = FreqDist(eng_tri)
fre_tri_freq = FreqDist(fre_tri)

To find bigram probabilities, we find the product of the individual trigram probabilities in the word. The individual trigram probabilities can be calculated by taking the the frequency of the trigram in the testing data and dividing it by the bigram frequency of the first character.<br>
Additionally, to account for unknown trigrams, Laplace or Add-One smoothing is performed. To do this, the numerator is increased by one, and the size of the vocabulary (number of unique characters in the training data) is added to the denominator<br>

In [22]:
#Function to find the trigram probabilities of data
def find_tri_prob(uni, freq_bi, freq_tri, test):
    v = len(set(uni))
    probs = []
    for word in test:
        word = word.lower()
        word = "__"+word+"__"
        word_prob = 1
        for c in range(len(word)-2):
            #word_prob *= (freq_tri.freq(word[c:c+3])+1)/(freq_bi.freq(word[c:c+2])+v)
            word_prob *= ((freq_tri[word[c:c+3]]+1)/(freq_bi[word[c:c+2]]+v))
        probs.append(word_prob)
    return probs

In [23]:
#Finding the trigram probabilities of the test data
eng_tri_prob = find_tri_prob(eng_uni, eng_bi_freq, eng_tri_freq, eng_test)
fre_tri_prob = find_tri_prob(fre_uni, fre_bi_freq, fre_tri_freq, eng_test)

In [24]:
#Comparing the probabilities of each word in the test set, creating scores for the models and calculating accuracies
eng_tri_score = fre_tri_score = 0
for i in range(len(eng_tri_prob)):
    if eng_tri_prob[i]>=fre_tri_prob[i]:
        eng_tri_score+=1
    else:
        fre_tri_score+=1

eng_tri_acc = (eng_tri_score/len(eng_tri_prob))*100
fre_tri_acc = (fre_tri_score/len(fre_tri_prob))*100

In [25]:
#Reporting accuracy of English vs. French Unigram Models
print('English Unigram Model Accuracy:', eng_uni_acc)
print('French Unigram Model Accuracy:', fre_uni_acc)

#Reporting accuracy of English vs. French Bigram Models
print('English bigram Model Accuracy:', eng_bi_acc)
print('French bigram Model Accuracy:', fre_bi_acc)

#Reporting accuracy of English vs. French Triigram Models
print('English trigram Model Accuracy:', eng_tri_acc)
print('French trigram Model Accuracy:', fre_tri_acc)

English Unigram Model Accuracy: 68.10000000000001
French Unigram Model Accuracy: 31.900000000000002
English bigram Model Accuracy: 74.1
French bigram Model Accuracy: 25.900000000000002
English trigram Model Accuracy: 90.10000000000001
French trigram Model Accuracy: 9.9


The above results show how model accuracy increases as the number of grams (N) increases.

<h2>Problem 2: Language Model Comparison<h2>

<h4>Creating similar language models to distinguish between Italian vs. Spanish text<h4>

<h4>Using Italian Testing Data<h4>

In [26]:
#Unigram Classification

#Creating Unigrams
ita_uni = make_unigrams(ita_train)
spa_uni = make_unigrams(spa_train)

#Creating frequency distributions of the unigrams
ita_uni_freq = FreqDist(ita_uni)
spa_uni_freq = FreqDist(spa_uni)

#Finding the probabilities of the unigrams
ita_uni_prob = find_uni_prob(ita_uni_freq, ita_test)
spa_uni_prob = find_uni_prob(spa_uni_freq, ita_test)

#Finding the accuracy of the model
ita_uni_score = spa_uni_score = 0
for i in range(len(ita_uni_prob)):
    if ita_uni_prob[i]>spa_uni_prob[i]:
        ita_uni_score+=1
    else:
        spa_uni_score+=1

ita_uni_acc = (ita_uni_score/len(ita_uni_prob))*100
spa_uni_acc = (spa_uni_score/len(spa_uni_prob))*100

In [27]:
#Bigram Classification

#Creating Birgrams
ita_bi = make_bigrams(ita_train)
spa_bi = make_bigrams(spa_train)

#Creating frequency distributions of bigrams
ita_bi_freq = FreqDist(ita_bi)
spa_bi_freq = FreqDist(spa_bi)

#Finding the probabilities of the bigrams
ita_bi_prob = find_bi_prob(ita_uni, ita_uni_freq, ita_bi_freq, ita_test)
spa_bi_prob = find_bi_prob(spa_uni, spa_uni_freq, spa_bi_freq, ita_test)

#Finding the accuracy of the models
ita_bi_score = spa_bi_score = 0
for i in range(len(ita_bi_prob)):
    if ita_bi_prob[i]>spa_bi_prob[i]:
        ita_bi_score+=1
    else:
        spa_bi_score+=1

ita_bi_acc = (ita_bi_score/len(ita_bi_prob))*100
spa_bi_acc = (spa_bi_score/len(spa_bi_prob))*100

In [28]:
#Trigram Classification

#Creating Trigrams
ita_tri = make_bigrams(ita_train)
spa_tri = make_bigrams(spa_train)

#Creating frequency distributions of Trigrams
ita_tri_freq = FreqDist(ita_tri)
spa_tri_freq = FreqDist(spa_tri)

#Finding the probabilities of the Trigrams
ita_tri_prob = find_tri_prob(ita_uni, ita_bi_freq, ita_tri_freq, ita_test)
spa_tri_prob = find_tri_prob(spa_uni, spa_bi_freq, spa_tri_freq, ita_test)

#Finding the accuracy of the models
ita_tri_score = spa_tri_score = 0
for i in range(len(ita_tri_prob)):
    if ita_tri_prob[i]>spa_tri_prob[i]:
        ita_tri_score+=1
    else:
        spa_tri_score+=1

ita_tri_acc = (ita_tri_score/len(ita_tri_prob))*100
spa_tri_acc = (spa_tri_score/len(spa_tri_prob))*100

In [29]:
#Reporting accuracy of Italian vs. Spanish Unigram Models
print('Italian Unigram Model Accuracy:', ita_uni_acc)
print('Spanish Unigram Model Accuracy:', spa_uni_acc)

#Reporting accuracy of Italian vs. Spanish Unigram Models
print('Italian Bigram Model Accuracy:', ita_bi_acc)
print('Spanish Bigram Model Accuracy:', spa_bi_acc)

#Reporting accuracy of Italian vs. Spanish Unigram Models
print('Italian Trigram Model Accuracy:', ita_tri_acc)
print('Spanish Trigram Model Accuracy:', spa_tri_acc)

Italian Unigram Model Accuracy: 52.2
Spanish Unigram Model Accuracy: 47.8
Italian Bigram Model Accuracy: 61.199999999999996
Spanish Bigram Model Accuracy: 38.800000000000004
Italian Trigram Model Accuracy: 61.0
Spanish Trigram Model Accuracy: 39.0


<h4>Using Spanish Testing Data<h4>

In [30]:
#Unigram Classification

#Creating Unigrams
ita_uni = make_unigrams(ita_train)
spa_uni = make_unigrams(spa_train)

#Creating frequency distributions of the unigrams
ita_uni_freq = FreqDist(ita_uni)
spa_uni_freq = FreqDist(spa_uni)

#Finding the probabilities of the unigrams
ita_uni_prob = find_uni_prob(ita_uni_freq, spa_test)
spa_uni_prob = find_uni_prob(spa_uni_freq, spa_test)

#Finding the accuracy of the model
ita_uni_score = spa_uni_score = 0
for i in range(len(ita_uni_prob)):
    if ita_uni_prob[i]>spa_uni_prob[i]:
        ita_uni_score+=1
    else:
        spa_uni_score+=1

ita_uni_acc = (ita_uni_score/len(ita_uni_prob))*100
spa_uni_acc = (spa_uni_score/len(spa_uni_prob))*100

In [31]:
#Bigram Classification

#Creating Birgrams
ita_bi = make_bigrams(ita_train)
spa_bi = make_bigrams(spa_train)

#Creating frequency distributions of bigrams
ita_bi_freq = FreqDist(ita_bi)
spa_bi_freq = FreqDist(spa_bi)

#Finding the probabilities of the bigrams
ita_bi_prob = find_bi_prob(ita_uni, ita_uni_freq, ita_bi_freq, spa_test)
spa_bi_prob = find_bi_prob(spa_uni, spa_uni_freq, spa_bi_freq, spa_test)

#Finding the accuracy of the models
ita_bi_score = spa_bi_score = 0
for i in range(len(ita_bi_prob)):
    if ita_bi_prob[i]>spa_bi_prob[i]:
        ita_bi_score+=1
    else:
        spa_bi_score+=1

ita_bi_acc = (ita_bi_score/len(ita_bi_prob))*100
spa_bi_acc = (spa_bi_score/len(spa_bi_prob))*100

In [32]:
#Trigram Classification

#Creating Trigrams
ita_tri = make_bigrams(ita_train)
spa_tri = make_bigrams(spa_train)

#Creating frequency distributions of Trigrams
ita_tri_freq = FreqDist(ita_tri)
spa_tri_freq = FreqDist(spa_tri)

#Finding the probabilities of the Trigrams
ita_tri_prob = find_tri_prob(ita_uni, ita_bi_freq, ita_tri_freq, spa_test)
spa_tri_prob = find_tri_prob(spa_uni, spa_bi_freq, spa_tri_freq, spa_test)

#Finding the accuracy of the models
ita_tri_score = spa_tri_score = 0
for i in range(len(ita_tri_prob)):
    if ita_tri_prob[i]>spa_tri_prob[i]:
        ita_tri_score+=1
    else:
        spa_tri_score+=1

ita_tri_acc = (ita_tri_score/len(ita_tri_prob))*100
spa_tri_acc = (spa_tri_score/len(spa_tri_prob))*100

In [33]:
#Reporting accuracy of Italian vs. Spanish Unigram Models
print('Italian Unigram Model Accuracy:', ita_uni_acc)
print('Spanish Unigram Model Accuracy:', spa_uni_acc)

#Reporting accuracy of Italian vs. Spanish Unigram Models
print('Italian Bigram Model Accuracy:', ita_bi_acc)
print('Spanish Bigram Model Accuracy:', spa_bi_acc)

#Reporting accuracy of Italian vs. Spanish Unigram Models
print('Italian Trigram Model Accuracy:', ita_tri_acc)
print('Spanish Trigram Model Accuracy:', spa_tri_acc)

Italian Unigram Model Accuracy: 22.7
Spanish Unigram Model Accuracy: 77.3
Italian Bigram Model Accuracy: 22.8
Spanish Bigram Model Accuracy: 77.2
Italian Trigram Model Accuracy: 96.3
Spanish Trigram Model Accuracy: 3.6999999999999997


With the above results we can see that differentiating between English and French documents is easier than differentiating between Italian and Spanish.<br>
While the models are generally capable to differentiate between Spanish and Italian, the margins by which they do this are not substantial. This is because Italian and Spanish are considered to be the most related out of all the romance languages.